Huggingface의 NLP tutorial 공부  
![huggingface](https://huggingface.co/front/assets/huggingface_logo-noborder.svg)

## 1. Huggingface의 의의

Transformer는 기본적으로 매우 큰 모델이다. 파라메터도 100억이 넘어서, 학습이나 배포가 매우 힘들다.  
특히, 새로운 모델이 거의 매일 나오고 있는데, 각각 개발자들 혹은 저자들이 구현한 것들을 모두 파악하고 실행해보는건 쉬운일이 아니다.

Transformer는 이러한 문제를 해결하기 위해 나온 library이다. 목표는 단일 API를 통해서 어떤 trasnformer 모델이던 불러와서 학습하고 저장할 수 있도록 하는 것.
- 사용하기 쉽고, flexible하면서 간단하게 하는 것.

## 2. Behind the pipeline (https://huggingface.co/learn/nlp-course/chapter2/2?fw=pt)

Chapter 1에서는 Pipeline이 자동적으로 input부터 prediction까지 전부 처리해주었지만, 어떻게 동작하는지를 파악하지 못하였다.  
이번 Section에서는 Pipeline의 작동 과정을 파악한다.  

1. Tokenizer: raw text를 모델이 받아들일 수 있는 numerical numbers로 바꾼다. 이를 위해 Tokenizer를 이용한다.
2. Model: Token들은 모델을 거쳐 Logits로 변환된다.
3. Post-processing: postprocessing을 통해 logits가 label과 score로 변한다.

![Transformer](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg)  
<sup>출처: https://huggingface.co/learn/nlp-course/chapter2/2?fw=pt </sup>

### 2-1. Tokenziation.

1. 먼저 raw text를 단어 단위나 특정 기준을 따라 분리한다.
2. 여기에 사용되는 모델에서 요구되는 special token들을 추가한다. (예: CLS, SEP)
3. Token들은 pre-trained model에서 사용되었던, 각 token에 부여되는 고유의 ID 값을 가진다.

Transformer 라이브러리는 AutoTokenizer라는 API를 제공한다.

In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life",
    "I hate this so much!"
]

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer의 결과인 `input`의 결과를 보면

In [ ]:
inputs

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}

- 두 문장의 길이가 다르기 때문에, 짧은 쪽에 padding을 해준다. 이는 padding=True 인자를 통해 이루어졌다.
- truncation = True는 모델이 다룰 수 있는 최대 길이 sequence를 넘어가면 max length까지만 보고 그 뒤는 안보겠다는 의미이다.
- return_tensors = "pt"는 tokenizer가 pytorch의 tensor형태로 변환되도록 하는 것이다.
- 결과 dictionary에서 'attention_mask"는 어디서 padding이 되었는지 알려주는 결과이다.

### 2-2. Model

Tokenizer처럼, AutoModel이 있다.

In [ ]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

torch.Size([2, 15, 768])


- 2는 batch-size, 15는 sequence length, 768은 각 모델 input의 vector dimension이다.

AutoModel이 model weight뿐만 아니라 configuration들도 가져오지만, model의 body만 instantiate를 한다고 한다.   

Output을 얻기 위해, 여기서는 classification task를 다루기에, AutoModelForSequenceClassification을 사용해야한다고 한다.

In [ ]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs.logits)

tensor([[-1.4683,  1.5105],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


- 하나는 positive label에 대한 score, 하나는 negative label에 대한 score이다.
- 아직 probability로 만들지는 않았다. 왜냐하면 각 transformer model이 logits을 반환하기 때문
- probability로의 변환은 Postprocessing 단계에서 softmax를 통해 진행한다.

### 2-3. Postprocessing

여기서는 softmax로 확률화를 만든다.